# Structure of a ConvLSTM model

### Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Define the ConvLSTM Model

The ConvLSTMCell class implements a single ConvLSTM unit, which processes input frame sequences one time step at a time.

In [ ]:
class ConvLSTMCell(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, kernel_size, bias): #input_dim = Number of channels in input, hidden_dim = Number of channels in hidden state, kernel_size = Size of the convolutional kernel, bias = If False, then the layer does not use bias weights b_ih and b_hh. Default
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias
        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim, # To compute all gates at once
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state # Unpack the current hidden and cell states
        combined = torch.cat([input_tensor, h_cur], dim=1) # Concatenate input with previous hidden state
        combined_conv = self.conv(combined) # Apply convolutional layer
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1) # The output of the convolution is split into four equal parts, each corresponding to one of the LSTM gates.
        i = torch.sigmoid(cc_i) # Input gate
        f = torch.sigmoid(cc_f) # Forget gate
        o = torch.sigmoid(cc_o) # Output gate
        g = torch.tanh(cc_g) # Candidate cell state
        c_next = f * c_cur + i * g # Compute the next cell state
        h_next = o * torch.tanh(c_next) # Compute the next hidden state
        return h_next, c_next

    def init_hidden(self, batch_size, image_size): # Creates tensors filled with zeros for both h_0 and c_0
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device), 
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module): # This class stacks multiple ConvLSTMCell layers to process entire sequences.
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False): # num_layers = Number of layers in the network
        super(ConvLSTM, self).__init__()
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers) # Allows specifying different kernel sizes and hidden dimensions per layer.
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers
        cell_list = []
        for i in range(0, self.num_layers): #Creates a list of ConvLSTMCell layers. The first layer receives input_dim, while subsequent layers receive the previous layer’s hidden_dim.
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]
            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))
        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        if not self.batch_first: #Ensures the input shape follows (batch, seq, channels, height, width).
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)
        b, _, _, h, w = input_tensor.size()
        if hidden_state is None:
            hidden_state = self._init_hidden(batch_size=b, image_size=(h, w)) #If no initial state is given, it initializes zeros.
        layer_output_list = []
        last_state_list = []
        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor
        for layer_idx in range(self.num_layers):
            h, c = hidden_state[layer_idx] # Get initial hidden and cell states for the current layer
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :], cur_state=[h, c])
                output_inner.append(h)
            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output
            layer_output_list.append(layer_output)
            last_state_list.append([h, c])
        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]
        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size): #Calls init_hidden for each layer, ensuring they are properly initialized.
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _extend_for_multilayer(param, num_layers): #Ensures that the kernel size and hidden dimensions are lists of length num_layers.
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

## Prepare the Dataset

In [ ]:
# Load dataset (example: MNIST-like sequences)
data = np.load("path_to_your_data.npy")  # Shape: (num_sequences, time_steps, channels, height, width)
data = torch.tensor(data, dtype=torch.float32) / 255.0  # Normalize to [0, 1]

# Split into train and test
train_data, test_data = data[:800], data[800:]

# Create PyTorch datasets
train_dataset = TensorDataset(train_data[:, :-1], train_data[:, -1])  # Input: first T-1 frames, Target: last frame
test_dataset = TensorDataset(test_data[:, :-1], test_data[:, -1])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

### Set Up Training

This code initializes and trains a ConvLSTM model using PyTorch

In [ ]:
# Initialize model, loss, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvLSTM(input_dim=1, hidden_dim=64, kernel_size=(3, 3), num_layers=2, batch_first=True).to(device)
criterion = nn.MSELoss() # MSE is commonly used for regression tasks, which is suitable for predicting pixel intensities in images
optimizer = optim.Adam(model.parameters(), lr=0.001) # Uses the Adam optimizer with a learning rate (lr) of 0.001 to update model weights

# Training loop The model is trained for 10 epochs. Each epoch means the model sees the entire training dataset once.
num_epochs = 10
for epoch in range(num_epochs):
    model.train() #Sets the model to training mode (enables dropout, batch normalization updates, etc.)
    total_loss = 0 #Initializes a variable to accumulate the total loss for the epoch
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() #Resets the gradients before backpropagation. By default, gradients accumulate in PyTorch, so this step prevents incorrect updates.
        outputs, _ = model(inputs) #Passes the input sequence through the model to get the output
        outputs = outputs[0][:, -1]  # Take the last output frame
        loss = criterion(outputs, targets)
        loss.backward() #Computes gradients of the loss with respect to model parameters
        optimizer.step() #Updates the model’s parameters using the gradients. This step is where the actual learning happens.
        total_loss += loss.item() #Accumulates the loss for the current batch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

### Validation


In [ ]:
from skimage.metrics import structural_similarity as ssim

def validate(model, loader, criterion, device):
    model.eval()
    total_loss, total_mae, total_rmse, total_ssim = 0, 0, 0, 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs, _ = model(inputs)
            outputs = outputs[0][:, -1]

            loss = criterion(outputs, targets)
            total_loss += loss.item()

            mae = torch.abs(outputs - targets).mean().item()
            rmse = torch.sqrt(((outputs - targets) ** 2).mean()).item()
            
            # Convert to numpy for SSIM
            output_np = outputs.cpu().numpy()
            target_np = targets.cpu().numpy()
            ssim_score = ssim(output_np, target_np, data_range=1.0, multichannel=True)
            
            total_mae += mae
            total_rmse += rmse
            total_ssim += ssim_score

    avg_loss = total_loss / len(loader)
    avg_mae = total_mae / len(loader)
    avg_rmse = total_rmse / len(loader)
    avg_ssim = total_ssim / len(loader)

    print(f"Validation Loss: {avg_loss:.4f}, MAE: {avg_mae:.4f}, RMSE: {avg_rmse:.4f}, SSIM: {avg_ssim:.4f}")
    return avg_loss, avg_mae, avg_rmse, avg_ssim

# Validate after each epoch
validate(model, test_loader, criterion, device)